# Notebook 1: Extract Z-scored LFP

Brief 1-2 sentence description of notebook.

In [1]:
# Imports of all used packages and libraries
import os
import glob
import sys


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
import spikeinterface.extractors as se
import spikeinterface.preprocessing as sp

In [4]:
# setting path
sys.path.append('../../src')

In [5]:
import trodes.read_exported

## Inputs & Data

Explanation of each input and where it comes from.

In [6]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [7]:
CHANNEL_MAPPING_DF = pd.read_excel("../../data/channel_mapping.xlsx")
TONE_TIMESTAMP_DF = pd.read_excel("../../data/rce_tone_timestamp.xlsx", index_col=0)

In [8]:
TONE_DIN = "dio_ECU_Din1"
TONE_STATE = 1

In [9]:
EPHYS_SAMPLING_RATE = 20000
LFP_SAMPLING_RATE = 1000
TRIAL_DURATION = 10
FRAME_RATE = 22
ECU_STREAM_ID = "ECU"
TRODES_STREAM_ID = "trodes"
LFP_FREQ_MIN = 0.5
LFP_FREQ_MAX = 300
ELECTRIC_NOISE_FREQ = 60
RECORDING_EXTENTION = "*.rec"

In [10]:
ALL_SESSION_DIR = glob.glob("/scratch/back_up/reward_competition_extention/data/omission/2023_06_17/*.rec")

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

# Functions

In [11]:
def compute_sorted_index(group, value_column='Value', index_column='SortedIndex'):
    """ 
    Computes the index of each row's value within its sorted group.

    Parameters:
    - group (pd.DataFrame): A group of data.
    - value_column (str): Name of the column containing the values to be sorted.
    - index_column (str): Name of the new column that will contain the indices.

    Returns:
    - pd.DataFrame: The group with an additional column containing the indices.
    """
    sorted_values = sorted(list(set(group[value_column].tolist())))
    group[index_column] = group[value_column].apply(lambda x: sorted_values.index(x))
    return group

In [12]:
def find_closest_index(sorted_list=None, target=0):
    """
    Returns the index of the number in the sorted list that is closest to the target.

    This function performs a binary search on a sorted list to find the closest number to 
    a given target. If the target exists in the list, its index is returned. If not, the 
    function will return the index of the number that's closest to the target.

    Parameters:
    - sorted_list (list[int or float]): A sorted list of numbers.
    - target (int or float): The target number to find the closest value to.

    Returns:
    - int: The index of the closest number in the sorted list to the target. 
           If the sorted list is empty, returns None.

    Example:
    >>> sorted_nums = [1, 3, 5, 8, 10, 15, 18, 20, 24, 27, 30]
    >>> find_closest_index(sorted_nums, 6)
    2

    Note:
    The list should be sorted in ascending order.
    """
    
    if sorted_list is None:
        return None
    if target <= sorted_list[0]:
        return 0
    if target >= sorted_list[-1]:
        return len(sorted_list) - 1

    # Binary search
    left, right = 0, len(sorted_list) - 1
    while left <= right:
        mid = (left + right) // 2

        if sorted_list[mid] == target:
            return mid
        elif sorted_list[mid] < target:
            left = mid + 1
        else:
            right = mid - 1

    # After binary search, the target will be between sorted_list[right] and sorted_list[left]
    # We compare the two to see which one is closer to the target and return its index
    if abs(sorted_list[left] - target) < abs(sorted_list[right] - target):
        return left
    else:
        return right

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [13]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# Ideally functions are defined here first and then data is processed using the functions

# function names are short and in snake case all lowercase
# a function name should be unique but does not have to describe the function
# doc strings describe functions not function names




# Electrophysiology

## Getting timestamps for each spikegadgets sample

In [14]:
session_to_dir = {}
# Going through each session recording
# Which includes all the recordings from all the miniloggers and cameras
for session_path in ALL_SESSION_DIR:   
    try:
        session_basename = os.path.splitext(os.path.basename(session_path))[0]
        print("Current Session: {}".format(session_basename))

        session_to_dir[session_basename] = trodes.read_exported.organize_all_trodes_export(session_path)
    except Exception as e: 
        print(e)

Current Session: 20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2
Skipping file 20230617_115521_standard_comp_to_omission_D1_subj_1-2_t2b2L_box2_merged.timestampoffset.txt due to error: Settings format not supported
Skipping file 20230617_115521_standard_comp_to_omission_D1_subj_1-1_t1b3L_box1_merged.timestampoffset.txt due to error: Settings format not supported


/nancy/projects/reward_competition_extention/results/2023_11_03_social_neuro_export/../../src/trodes/read_exported.py:62: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  return np.dtype(dtype_spec)


Current Session: 20230617_115641_standard_comp_to_omission_D1_subj_2-2_and_2-4


In [47]:
session_to_recording_to_timestamps = []
for dir, rec_dict in session_to_dir.items():
    for rec_file, value in rec_dict.items():
        voltage_timestamp_array = np.array([lst[0] for lst in np.array(value["raw"]["timestamps"]["data"])])
        voltage_index_array = voltage_timestamp_array - voltage_timestamp_array[0]
        
        session_to_recording_to_timestamps.append({"recording_session": dir, "ephys_name": rec_file, "voltage_timestamps": voltage_timestamp_array, "voltage_indexes": voltage_index_array})

In [48]:
recording_sessions_df = pd.DataFrame(session_to_recording_to_timestamps)

In [49]:
recording_sessions_df["subject_id"] = recording_sessions_df["ephys_name"].apply(lambda x: x.split("subj")[-1].strip("_").split("_")[0].replace("-", "."))

In [50]:
recording_sessions_df["subject_id"] 

0    1.1
1    1.2
Name: subject_id, dtype: object

In [51]:
recording_sessions_df["ephys_name"].iloc[0]

'20230617_115521_standard_comp_to_omission_D1_subj_1-1_t1b3L_box1_merged'

- Adding the ephys channels

In [52]:
CHANNEL_MAPPING_DF

,Cohort,Subject,eib_mPFC,eib_vHPC,eib_BLA,eib_LH,eib_MD,spike_interface_mPFC,spike_interface_vHPC,spike_interface_BLA,spike_interface_LH,spike_interface_MD
0,1,6.1,NaN,15,14,13,31,21.0,15.0,14.0,13.0,16.0
1,1,6.2,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
2,1,6.3,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
3,1,6.4,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
4,2,1.1,NaN,16,17,18,19,5.0,31.0,30.0,29.0,28.0
5,2,1.2,NaN,31,30,29,28,10.0,31.0,30.0,29.0,28.0
6,2,1.3,NaN,15,14,13,12,9.0,31.0,30.0,29.0,28.0
7,2,1.4,NaN,15,14,13,12,15.0,31.0,30.0,29.0,28.0


- Adding all the brain region to ch information

In [53]:
CHANNEL_MAPPING_DF["Subject"] = CHANNEL_MAPPING_DF["Subject"].astype(str)

In [54]:
recording_sessions_df = pd.merge(left=recording_sessions_df, left_on="subject_id", right=CHANNEL_MAPPING_DF, right_on="Subject")

In [55]:
recording_sessions_df

,recording_session,ephys_name,voltage_timestamps,voltage_indexes,subject_id,Cohort,Subject,eib_mPFC,eib_vHPC,eib_BLA,eib_LH,eib_MD,spike_interface_mPFC,spike_interface_vHPC,spike_interface_BLA,spike_interface_LH,spike_interface_MD
0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,"[3478533, 3478534, 3478535, 3478536, 3478537, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",1.1,2,1.1,NaN,16,17,18,19,5.0,31.0,30.0,29.0,28.0
1,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,"[3478533, 3478534, 3478535, 3478536, 3478537, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",1.2,2,1.2,NaN,31,30,29,28,10.0,31.0,30.0,29.0,28.0


In [56]:
recording_sessions_df["ephys_name"].iloc[0]

'20230617_115521_standard_comp_to_omission_D1_subj_1-1_t1b3L_box1_merged'

# Extracting the LFP

In [57]:
recording_name_to_all_ch_lfp = {}
# Going through all the recording sessions 
for session_dir in ALL_SESSION_DIR:
    # Going through all the recordings in each session
    for recording_path in glob.glob(os.path.join(session_dir, RECORDING_EXTENTION)):
        try:
            recording_basename = os.path.splitext(os.path.basename(recording_path))[0]
            # checking to see if the recording has an ECU component
            # if it doesn't, then the next one be extracted
            current_recording = se.read_spikegadgets(recording_path, stream_id=ECU_STREAM_ID)
            current_recording = se.read_spikegadgets(recording_path, stream_id=TRODES_STREAM_ID)
            print(recording_basename)
            # Preprocessing the LFP
            current_recording = sp.bandpass_filter(current_recording, freq_min=LFP_FREQ_MIN, freq_max=LFP_FREQ_MAX)
            current_recording = sp.notch_filter(current_recording, freq=ELECTRIC_NOISE_FREQ)
            current_recording = sp.resample(current_recording, resample_rate=LFP_SAMPLING_RATE)
            current_recording = sp.zscore(current_recording)
            recording_name_to_all_ch_lfp[recording_basename] = current_recording
        except Exception as error:
            # handle the exception
            print("An exception occurred:", error) # An exception occurred: division by zero




An exception occurred: stream_id trodes is not in ['ECU']
20230617_115521_standard_comp_to_omission_D1_subj_1-2_t2b2L_box2_merged
20230617_115521_standard_comp_to_omission_D1_subj_1-1_t1b3L_box1_merged
An exception occurred: stream_id trodes is not in ['ECU']


In [59]:
recording_sessions_df["all_ch_lfp"] = recording_sessions_df["ephys_name"].map(recording_name_to_all_ch_lfp)

In [60]:
recording_sessions_df

,recording_session,ephys_name,voltage_timestamps,voltage_indexes,subject_id,Cohort,Subject,eib_mPFC,eib_vHPC,eib_BLA,eib_LH,eib_MD,spike_interface_mPFC,spike_interface_vHPC,spike_interface_BLA,spike_interface_LH,spike_interface_MD,all_ch_lfp
0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,"[3478533, 3478534, 3478535, 3478536, 3478537, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",1.1,2,1.1,NaN,16,17,18,19,5.0,31.0,30.0,29.0,28.0,ZScoreRecording: 32 channels - 1.0kHz - 1 segm...
1,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,"[3478533, 3478534, 3478535, 3478536, 3478537, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",1.2,2,1.2,NaN,31,30,29,28,10.0,31.0,30.0,29.0,28.0,ZScoreRecording: 32 channels - 1.0kHz - 1 segm...


- Getting the LFP for each brain region

In [61]:
region_columns = [col for col in recording_sessions_df if "spike_interface" in col]

In [62]:
region_columns

['spike_interface_mPFC',
 'spike_interface_vHPC',
 'spike_interface_BLA',
 'spike_interface_LH',
 'spike_interface_MD']

In [63]:
for col in region_columns:
    recording_sessions_df[col] = recording_sessions_df[col].astype(int).astype(str)
    region = col.split("_")[-1]
    print(region)
    recording_sessions_df["{}_lfp_trace".format(region)] = recording_sessions_df.apply(lambda row: row["all_ch_lfp"].get_traces(channel_ids=[row[col]]).T[0], axis=1)

mPFC
vHPC
BLA
LH
MD


In [65]:
trace_columns = [col for col in recording_sessions_df if "trace" in col]

In [67]:
recording_sessions_df["lfp_indexes"] = recording_sessions_df[trace_columns[0]].apply(lambda x: np.arange(0, 20 * x.shape[0] + 1, 20))

In [69]:
recording_sessions_df["lfp_indexes"].iloc[0]

array([       0,       20,       40, ..., 79697460, 79697480, 79697500])

In [71]:
recording_sessions_df["voltage_indexes"].iloc[0]

array([       0,        1,        2, ..., 79698583, 79698584, 79698585],
      dtype=uint32)

In [72]:
recording_sessions_df

,recording_session,ephys_name,voltage_timestamps,voltage_indexes,subject_id,Cohort,Subject,eib_mPFC,eib_vHPC,eib_BLA,...,spike_interface_BLA,spike_interface_LH,spike_interface_MD,all_ch_lfp,mPFC_lfp_trace,vHPC_lfp_trace,BLA_lfp_trace,LH_lfp_trace,MD_lfp_trace,lfp_indexes
0,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,"[3478533, 3478534, 3478535, 3478536, 3478537, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",1.1,2,1.1,NaN,16,17,...,30,29,28,ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0.07365118, 0.11047677, 0.18994251, 0.2364590...","[-0.26256362, -0.19967411, -0.015722372, 0.150...","[0.28542638, 0.8104071, 1.2419446, 1.0261759, ...","[0.0040029064, -0.11808574, -0.052037787, 0.17...","[0.103898674, -0.011922799, 0.020439083, 0.136...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2..."
1,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,"[3478533, 3478534, 3478535, 3478536, 3478537, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",1.2,2,1.2,NaN,31,30,...,30,29,28,ZScoreRecording: 32 channels - 1.0kHz - 1 segm...,"[0.9489666, 1.1787612, 1.587285, 1.6766496, 1....","[-0.59670866, -0.5804559, -0.500353, -0.304158...","[0.57716405, 1.2267566, 1.8559785, 1.9170899, ...","[1.0252244, 1.567814, 1.8645895, 1.6457549, 1....","[0.8103085, 0.9092951, 0.8448387, 0.6675837, 0...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2..."


In [64]:
recording_sessions_df["mPFC_lfp_trace"].iloc[0].shape

(3984875,)

In [36]:
np.arange(0, 20 * recording_sessions_df["mPFC_lfp_trace"].iloc[0].shape[0] + 1, 20)

array([       0,       20,       40, ..., 79697460, 79697480, 79697500])

- Getting the timestamps of each LFP sample

In [ ]:
rec

In [33]:
raise ValueError()

ValueError: 

In [ ]:
region

In [ ]:
pd.DataFrame.from_dict(recording_name_to_all_ch_lfp)

In [ ]:
79698586/20

In [ ]:
raise ValueError()

In [ ]:
recording_name_to_all_ch_lfp["20230617_115521_standard_comp_to_omission_D1_subj_1-1_t1b3L_box1_merged"].get_traces()

- Filtering for all trials that we got the LFP for

In [ ]:
all_trials_df = all_trials_df[all_trials_df["recording_file"].isin(recording_name_to_all_ch_lfp.keys())].reset_index(drop=True)

In [ ]:
all_trials_df.head()

- Adding trial numbers based on timestamp ordering for each recording

In [ ]:
all_trials_df = all_trials_df.groupby('recording_file').apply(lambda g: compute_sorted_index(g, value_column='time', index_column='trial_number')).reset_index(drop=True)

In [ ]:
all_trials_df["trial_number"] = all_trials_df["trial_number"] + 1

## Adding the LFP trace information

In [ ]:
channel_map_and_all_trials_df = all_trials_df.merge(CHANNEL_MAPPING_DF, left_on="current_subject", right_on="Subject", how="left")

In [ ]:
channel_map_and_all_trials_df = channel_map_and_all_trials_df.drop(columns=[col for col in channel_map_and_all_trials_df.columns if "eib" in col], errors="ignore")

In [ ]:
channel_map_and_all_trials_df = channel_map_and_all_trials_df.drop(columns=["Subject"], errors="ignore")

In [ ]:
channel_map_and_all_trials_df.head()

In [ ]:
channel_map_and_all_trials_df.to_csv("./proc/trial_metadata.csv")

In [ ]:
channel_map_and_all_trials_df.to_pickle("./proc/trial_metadata.pkl")

In [ ]:
channel_map_and_all_trials_df.columns

- Linking up all LFP calculations with all the trials

In [ ]:
channel_map_and_all_trials_df["all_ch_lfp"] = channel_map_and_all_trials_df["recording_file"].map(recording_name_to_all_ch_lfp)

- Creating a new row for each brain region

In [ ]:
brain_region_col = [col for col in CHANNEL_MAPPING_DF if "spike_interface" in col]

In [ ]:
id_cols = [col for col in channel_map_and_all_trials_df.columns if col not in brain_region_col]

In [ ]:
brain_region_col

In [ ]:
for col in brain_region_col:
    channel_map_and_all_trials_df[col] = channel_map_and_all_trials_df[col].astype(int).astype(str)

In [ ]:
channel_map_and_all_trials_df.columns

In [ ]:
for col in brain_region_col:
    print(col)
    channel_map_and_all_trials_df["{}_baseline_lfp_trace".format(col.strip("spike_interface").strip("_"))] = channel_map_and_all_trials_df.apply(lambda row: row["all_ch_lfp"].get_traces(channel_ids=[row[col]], start_frame=row["baseline_lfp_timestamp_range"][0], end_frame=row["baseline_lfp_timestamp_range"][1]).T[0], axis=1)

    channel_map_and_all_trials_df["{}_trial_lfp_trace".format(col.strip("spike_interface").strip("_"))] = channel_map_and_all_trials_df.apply(lambda row: row["all_ch_lfp"].get_traces(channel_ids=[row[col]], start_frame=row["trial_lfp_timestamp_range"][0], end_frame=row["trial_lfp_timestamp_range"][1]).T[0], axis=1)


In [ ]:
channel_map_and_all_trials_df = channel_map_and_all_trials_df.drop(columns=["all_ch_lfp"], errors="ignore")

In [ ]:
channel_map_and_all_trials_df.to_pickle("./proc/full_baseline_and_trial_lfp_traces.pkl")

In [ ]:
channel_map_and_all_trials_df.head()

In [ ]:
channel_map_and_all_trials_df.columns

In [ ]:
(83177118-3478533)/20000

# LOOP 2: Extracting the timestamps for the raw ephys recording

In [ ]:
session_to_din_state_df = {}
for session, file_to_data in session_to_dir.items():
    all_recording_din_state_df = []
    for recording_name, subdir_dict in file_to_data.items():
        print(recording_name)
        current_recording_din_state_df = []

        try:
            voltage_timestamp_array = file_to_data[recording_name]["raw"]["timestamps"]["data"]
            for key, value in file_to_data[recording_name]["DIO"].items():
                if "in" in key:
                    print(key)
                    din_state_array = file_to_data[recording_name]["DIO"][key]["data"]
                    current_din_state_df = pd.DataFrame(din_state_array)
                    current_din_state_df["recording_dir"] = session
                    current_din_state_df["recording_file"] = recording_name
                    current_din_state_df["din"] = key
                    current_recording_din_state_df.append(current_din_state_df)
                    if key == TONE_DIN:
                        plt.plot([tup[0] for tup in din_state_array], [tup[1] for tup in din_state_array])
                        plt.xlabel("Timestamp")
                        plt.ylabel("State")
                        plt.title("Din State Change against Timestamps for {} in {}".format(key, recording_name))
                        plt.show()
                        plt.close()
            concatted_per_recording_din_state_df = pd.concat(current_recording_din_state_df).sort_values(by=["recording_file", "din"]).reset_index(drop=True)
            concatted_per_recording_din_state_df["time_stamp_index"] = concatted_per_recording_din_state_df["time"] - voltage_timestamp_array[0][0]
            all_recording_din_state_df.append(concatted_per_recording_din_state_df)
        except Exception as e: 
            print(e)
    concatted_all_recording_din_state_df = pd.concat(all_recording_din_state_df)
    session_to_din_state_df[session] = concatted_all_recording_din_state_df

In [ ]:
voltage_timestamp_array.shape

In [ ]:
79698586//20000

In [ ]:
concatted_per_recording_din_state_df

In [ ]:
all_recording_din_state_df

# LOOP 3 Adding the video timestamps

In [ ]:
session_to_din_state_df.keys()

In [ ]:
session_to_din_with_frames_df = {}
for session_path in all_session_files:   
    try:
        session_basename = os.path.splitext(os.path.basename(session_path))[0]
        print("Current Session: {}".format(session_basename))
        file_to_video_timestamps = {}
        for video_timestamps in glob.glob(os.path.join(session_path, "*cameraHWSync")):
            video_basename = os.path.basename(video_timestamps)
            print("Current Video Name: {}".format(video_basename))
            timestamp_array = trodes.read_exported.read_trodes_extracted_data_file(video_timestamps)["data"]["PosTimestamp"]
            file_to_video_timestamps[video_basename] = timestamp_array
            session_to_din_state_df[session_basename][os.path.basename(video_timestamps)] = session_to_din_state_df[session_basename]["time"].apply(lambda x: find_closest_index(sorted_list=timestamp_array, target=x))        
        
        # Find the maximum length of the arrays in the dictionary
        max_length = max(map(len, file_to_video_timestamps.values()))
        
        # Pad each array with NaN values to make them all the same length
        padded_data = {k: np.pad(v, (0, max_length - len(v)), mode='constant', constant_values=np.nan) for k, v in file_to_video_timestamps.items()}
        
        # Convert the padded data to a dataframe
        session_to_din_with_frames_df[session_basename] = pd.DataFrame(padded_data)
        session_to_din_with_frames_df[session_basename].to_csv(os.path.join(OUTPUT_DIR, "{}.frame_to_timestamps.csv".format(session_basename)))
    except Exception as e: 
        print(e)

session_to_din_state_df['20230612_101430_standard_comp_to_training_D1_subj_1-4_and_1-3'].head()

session_to_din_with_frames_df['20230612_101430_standard_comp_to_training_D1_subj_1-4_and_1-3'].head()

# LOOP 4: Combining the video columns

In [ ]:
session_to_tone_stamp_df = {}
for session, timestamps_df in session_to_din_state_df.items():
    current_timestamps_df = timestamps_df[(timestamps_df["din"] == TONE_DIN) & (timestamps_df["state"] == TONE_STATE)].reset_index(drop=True)
    camera_col = [col for col in current_timestamps_df.columns if "cameraHWSync" in col]
    id_col = [col for col in current_timestamps_df.columns if "cameraHWSync" not in col]
    
    current_timestamps_df = current_timestamps_df.melt(id_vars=id_col, value_vars=camera_col, var_name='video_file', value_name='video_frame')
    current_timestamps_df["video_number"] = current_timestamps_df["video_file"].apply(lambda x: x.strip("videoTimeStamps.cameraHWSync").split(".")[-1])
    current_timestamps_df["subject_info"] = current_timestamps_df["recording_file"].apply(lambda x: x.split("subject")[-1].strip("merged").strip("_"))
    current_timestamps_df["condition"] = np.nan
    session_to_tone_stamp_df[session]  = current_timestamps_df

In [ ]:
session_to_tone_stamp_df[session]

In [ ]:
concatenated_tone_stamp_df = pd.concat(session_to_tone_stamp_df.values()).reset_index(drop=True)

In [ ]:
concatenated_tone_stamp_df.head()

In [ ]:
concatenated_tone_stamp_df

In [ ]:
concatenated_tone_stamp_df.to_csv(os.path.join(OUTPUT_DIR, "{}_tone_timestamp.csv".format(OUTPUT_PREFIX)))

In [ ]:
[1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1]

# Reformatting Dataframe

- Dropping all rows that have not been labeled

In [ ]:
all_trials_df = TONE_TIMESTAMP_DF.dropna(subset="condition").reset_index(drop=True)

In [ ]:
sorted(all_trials_df["recording_dir"].unique())

- Making the video frame number usable

In [ ]:
all_trials_df["video_frame"] = all_trials_df["video_frame"].astype(int)

- Getting the name of the video so that we can sync it up with the ephys recording

In [ ]:
all_trials_df["video_name"]  = all_trials_df["video_file"].apply(lambda x: x.strip(".videoTimeStamps.cameraHWSync"))

- Getting all subject IDs for a given recording

In [ ]:
# using different id extractions for different file formats
all_trials_df["all_subjects"] = all_trials_df["recording_dir"].apply(lambda x: x if "2023" in x else "subj" + "_".join(x.split("_")[-5:]))
all_trials_df["all_subjects"] = all_trials_df["all_subjects"].apply(lambda x: tuple(sorted([num.strip("_").replace("_",".") for num in x.replace("-", "_").split("subj")[-1].strip("_").split("and")])))

In [ ]:
all_trials_df["all_subjects"].unique()

In [ ]:
all_trials_df["current_subject"] = all_trials_df["subject_info"].apply(lambda x: ".".join(x.replace("-","_").split("_")[:2])).astype(str)

In [ ]:
all_trials_df["current_subject"].unique()

- Converting the trial label to win or lose based on who won the trial

In [ ]:
all_trials_df["trial_outcome"] = all_trials_df.apply(
    lambda x: "win" if str(x["condition"]).strip() == str(x["current_subject"]) 
             else ("lose" if str(x["condition"]) in x["all_subjects"] 
                   else x["condition"]), axis=1)

In [ ]:
all_trials_df["trial_outcome"].unique()

- Adding the competition closeness as a column

In [ ]:
competition_closeness_map = {k: "non_comp" if "only" in str(k).lower() else "comp" if type(k) is str else np.nan for k in all_trials_df["competition_closeness"].unique()}

In [ ]:
competition_closeness_map

In [ ]:
all_trials_df["competition_closeness"] = all_trials_df["competition_closeness"].map(competition_closeness_map)

In [ ]:
all_trials_df["competition_closeness"] = all_trials_df.apply(lambda x: "_".join([str(x["trial_outcome"]), str(x["competition_closeness"])]).strip("nan").strip("_"), axis=1)

In [ ]:
all_trials_df["competition_closeness"].unique()

- Adding the LFP index

In [ ]:
all_trials_df["lfp_index"] = (all_trials_df["time_stamp_index"] // (EPHYS_SAMPLING_RATE/LFP_SAMPLING_RATE)).astype(int)

In [ ]:
all_trials_df["time"] = all_trials_df["time"].astype(int)

In [ ]:
all_trials_df["time_stamp_index"] = all_trials_df["time_stamp_index"].astype(int)

- Removing unnecessary columns

In [ ]:
all_trials_df = all_trials_df.drop(columns=["state", "din", "condition", "Unnamed: 13"], errors="ignore")

In [ ]:
all_trials_df.head()

In [ ]:
all_trials_df.groupby(["competition_closeness"]).count()

- Making columns of the different timestamps

In [ ]:
all_trials_df["baseline_lfp_timestamp_range"] = all_trials_df["lfp_index"].apply(lambda x: (x - TRIAL_DURATION * LFP_SAMPLING_RATE, x))

In [ ]:
all_trials_df["trial_lfp_timestamp_range"] = all_trials_df["lfp_index"].apply(lambda x: (x, x + TRIAL_DURATION * LFP_SAMPLING_RATE))

In [ ]:
all_trials_df["baseline_ephys_timestamp_range"] = all_trials_df["time_stamp_index"].apply(lambda x: (x - TRIAL_DURATION * EPHYS_SAMPLING_RATE, x))

In [ ]:
all_trials_df["trial_ephys_timestamp_range"] = all_trials_df["time_stamp_index"].apply(lambda x: (x, x + TRIAL_DURATION * EPHYS_SAMPLING_RATE))

In [ ]:
all_trials_df["baseline_videoframe_range"] = all_trials_df["video_frame"].apply(lambda x: (x - TRIAL_DURATION * FRAME_RATE, x))

In [ ]:
all_trials_df["trial_videoframe_range"] = all_trials_df["video_frame"].apply(lambda x: (x, x + TRIAL_DURATION * FRAME_RATE))